## 1. PP-TSM Introduction

Video classification is similar to image classification which belongs to the recognition task. For a given input video, the video classification model aims to output its predicted label category. If tags are all action categories, this task is also called action recognition. Different from image classification, video classification often requires the use of temporal information between multiple frames of images. PP-TSM is a practical industrial video classification model developed by PaddleVideo. Based on the implementation of state-of-the-art algorithms, we slim the model size and optimize the accuracy with the considerations of the trade-off between speed and precision.

PP-TSM is produced based on ResNet-50 backbone. Optimized methods includes data augmentation, network structure fine-tuning, training strategy, preciceBN, pretrain model selection and model distillation. Under the premise of basically not increasing the amount of calculation, using the center-sampling evaluation method, the accuracy of PP-TSM on Kinetics-400 is 3.95 points higher than that of the original paper, reaching 76.16%, which exceeds the 3D model under the same backbone network, and the inference speed is 4.5 times faster!

More information about PaddleVideo can be found here https://github.com/PaddlePaddle/PaddleVideo .



## 2. Model Effects and Application Scenarios
### 2.1 action recognition Tasks:

#### 2.1.1 Datasets：

The dataset is mainly in Kinetics-400, which is divided into training set and test set.

#### 2.1.2 Model Effects：

The recognition effect of PP-TSM on the picture is:

<div align="center">
<img src="https://user-images.githubusercontent.com/22365664/200510601-85b04d54-e892-43db-8a92-679f623bb6e4.gif"  width = "80%"  />
</div>




## 3. How to Use the Model

### 3.1 Model Inference:
* Download 

In [ ]:
%cd ~/work

!git clone https://github.com/PaddlePaddle/PaddleVideo

* Installation

In [ ]:
# The script needs to be run in the PaddleVideo directory
%cd ~/work/PaddleVideo/

# Install the required dependencies [already persisted, no need to install again].
!pip install --upgrade pip
!pip install -r requirements.txt --user

# Install PaddleVideo
!pip install ppvideo==2.3.0 --user

* Quick experience

Congratulations! Now that you've successfully installed PaddleVideo, let's get a quick feel at action recognition.

In [ ]:
!ppvideo --model_name='ppTSM' --use_gpu=False --video_file='data/example.avi'

An video with the predicted result is generated.

The result is as follows：

```txt
Current video file: data/example.avi
        top-1 classes: [5]
        top-1 scores: [0.95056254]
        top-1 label names: ['archery']
```

### 3.2 Model Training
* Clone the PaddleVideo repository (see 3.1 for details)

* Prepare the datasets and pretrain model.

In [ ]:
# 进入PaddleVideo工作目录
%cd ~/work/PaddleVideo/

# 下载Kinetics-400小数据集
%pushd ./data/k400
!wget -nc https://videotag.bj.bcebos.com/Data/k400_videos_small.tar
!tar -xf k400_videos_small.tar
%popd

# 下载预训练模型
!wget -nc -P ./data https://videotag.bj.bcebos.com/PaddleVideo/PretrainModel/ResNet50_vd_ssld_v2_pretrained.pdparams --no-check-certificate


* Change yaml configurations files.


Change yaml configurations files` configs/recognition/pptsm/pptsm_k400_videos_uniform.yaml`

```
DATASET: #DATASET field
    batch_size: 4  # adjust according to GPU memory
    num_workers: 4 
    test_batch_size: 1
    train:
        format: "VideoDataset" 
        data_prefix: "data/k400/videos" 
        file_path: "data/k400/train_small_videos.list " # modify train dataset path
    valid:
        format: "VideoDataset" 
        data_prefix: "data/k400/videos" 
        file_path: "data/k400/val_small_videos.list" #modify validation dataset path
    test:
        format: "VideoDataset"
        data_prefix: "data/k400/videos" 
        file_path: "data/k400/val_small_videos.list" #modify validation dataset path
```


* Train the model.

In [ ]:
%cd ~/work/PaddleVideo/
%env CUDA_VISIBLE_DEVICES=0

# Start training
!python main.py --validate -c configs/recognition/pptsm/pptsm_k400_videos_uniform.yaml

* Model evaluation

Setting `--validate` in train script, the evaluation will be implemented during training. For a trained model, the evaluation code as follows.

`-c` specify configuration file, `-w` specify checkpoints.

In [ ]:
%cd ~/work/PaddleVideo/
%env CUDA_VISIBLE_DEVICES=0

!python main.py --test -c configs/recognition/pptsm/pptsm_k400_videos_uniform.yaml -w output/ppTSM/ppTSM_best.pdparams

## 4. Model Principles

* Temporal Shift Module

PP-TSM use Temporal Shift Module to learn temporal information. This method greatly improves the model's ability to use the video information without adding any additional parameters or computation.

<div align="center">
<img src="https://raw.githubusercontent.com/PaddlePaddle/PaddleVideo/develop/docs/images/tsm_architecture.png" height=250 width=700 hspace='10'/> <br />
</div>


* Data Augmentation VideoMix

For Video Mix-up，The two videos are mixed with a certain weight value to form a new input video, which improves the space-time capability of the network.

* precise BN

In order to obtain more accurate mean and variance for BN layer to use in testing, in the experiment, we fix the parameters in the network after the network has trained an Epoch, then input the training data into the network for forward calculation, save the mean and square error of each step, and finally obtain the accurate mean and variance of all training samples to improve the testing accuracy.

## 5. Note

Each configuration file provided by the PP-TSM model is placed in the `configs/recognition/pptsm` directory. The configuration file name is organized in the following format:

```ModelName_Backbone_Datasest_DataFormat_EvaluationMethod_others.yaml```

* Data format includes frame and video, video indicates training with online video, frame indicates training with offline frames. Due to the differences caused by decoding, the models obtained from the training of this two formats may have some differences in accuracy.

* The evaluation methods include uniform and dense. Uniform means central sampling, and dense means dense sampling.


## 6. Related papers and citations

```
@inproceedings{lin2019tsm,
  title={TSM: Temporal Shift Module for Efficient Video Understanding},
  author={Lin, Ji and Gan, Chuang and Han, Song},
  booktitle={Proceedings of the IEEE International Conference on Computer Vision},
  year={2019}
} 
```
